Flow_1_Agents


In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.ai.generativelanguage_v1beta.types import Tool as GenAITool

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    google_api_key="",
    temperature=0.1,
    max_retries=3,
)



/home/kelvin/miniconda3/envs/genai-dev/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E0000 00:00:1762279858.386555  121906 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [ ]:

resp = llm.invoke(
    "Patient Aarav Mehta, a 29-year-old male, has Type 2 Diabetes. Dietary restrictions include Low sugar, No sweets. Admitted on November 03, 2025 - suggest a meal plan",
    tools=[GenAITool(google_search={})],
)

print(resp.content)

In [46]:
from langchain.prompts import PromptTemplate

Diet_Planing_Prompt = PromptTemplate(
    input_variables=["Patient_Info", "Recipes_List", "Dietitian_Notes"],
    template_format="jinja2",
    template="""
You are a clinical nutritionist tasked with creating a personalized daily diet plan for a patient.
### Instructions:
1. Create a diet plan for 5 meals: Breakfast, Mid-Morning Snack, Lunch, Evening Snack, Dinner.
2. Use only recipes from the recipe list unless absolutely necessary to create a new one.
3. If existing recipes do NOT fulfill the patient's requirements, flag a need to add new recipes.
4. RESPOND WITH RAW JSON ONLY. DO NOT include backticks, markdown formatting, comments, or any extra text.

### Patient Info:
{{ Patient_Info }}

{% if Dietitian_Notes %}
### Dietitian Notes:
{{ Dietitian_Notes }}
{% endif %}

### Existing Recipes (use these if they meet needs):
{{ Recipes_List }}

---

### IMPORTANT OUTPUT RULES:
- Output **ONLY** valid JSON.
- Do **NOT** include backticks.
- Do **NOT** include markdown formatting (like ```json).
- Do **NOT** add explanations, comments, headings, or extra text.
- The **FIRST character of the response MUST be `{`** and the **LAST must be `}`**.

---

### STRICT JSON FORMAT TO RETURN:
{
  "plan_desc": "Long Description of the diet plan.",
  "add_new_recipe": true or false,
  "new_recipes": [
    { "name": "Recipe Name", "desc": "1-sentence reason this recipe is needed." }
  ]
}

- If no new recipes are needed, set `"add_new_recipe": false` and `"new_recipes": []`.
"""
)


In [13]:
Patient_Info = "Patient Aarav Mehta, a 29-year-old male, diagnosed with Type 2 Diabetes. Dietary restrictions: Low sugar, No sweets. Admitted on November 03, 2025. Nutrition tracking since admission includes:  On November 01, 2025, the patient had a meal with an intake of 95.00% and blood sugar level of 128.50 mg/dL. Notes: Patient ate well, blood sugar within target range On November 01, 2025, the patient had a meal with an intake of 100.00% and blood sugar level of 142.00 mg/dL. Notes: Completed lunch, slight elevation in blood sugar On November 01, 2025, the patient had a meal with an intake of 85.00% and blood sugar level of 135.00 mg/dL. Notes: Left some vegetables, blood sugar acceptable"
Dietitian_Notes=""
Recipes_List = "1. Diabetic-Friendly Salmon Bowl – Baked salmon with quinoa and spinach, low glycemic index, 2. Gluten-Free Oatmeal Breakfast – Steel-cut oats with blueberries and low-fat milk, 3. Grilled Chicken with Steamed Vegetables – Lean grilled chicken breast served with steamed broccoli and carrots, 4. High-Protein Greek Yogurt Parfait – Greek yogurt layered with blueberries (nut-free), 5. Kidney-Friendly Lentil Soup – Low potassium lentil soup with carrots and herbs, 6. Low-Carb Tofu Scramble – Scrambled tofu with spinach and tomatoes, 7. Low-Sodium Turkey Stir-Fry – Turkey breast with mixed vegetables in olive oil, 8. Mediterranean Quinoa Salad – Quinoa with tomatoes, spinach, and olive oil dressing, 9. Pureed Sweet Potato Blend – Smooth pureed sweet potato for dysphagia patients, 10. Soft Diet Chicken Puree – Finely pureed chicken with gravy for post-surgical patients"

In [47]:
Diet_Planing_generator_chain = Diet_Planing_Prompt | llm
response = Diet_Planing_generator_chain.invoke(
    {
        "Patient_Info": Patient_Info,
        "Recipes_List": Recipes_List,
        "Dietitian_Notes":Dietitian_Notes if Dietitian_Notes is not None else None,
    },
    tools=[GenAITool(google_search={})],
)

In [ ]:
import json
import re

def clean_and_parse_json(llm_output: str):
    """
    Cleans common LLM artifacts like code fences and extra text, and parses clean JSON.
    """
    # Step 1: Remove surrounding whitespace
    
    cleaned = llm_output.strip()

    # Step 2: Remove markdown-style code fences (```json or ```)
    cleaned = re.sub(r"^```.*?(\n|$)", "", cleaned)  # Remove opening ```
    cleaned = re.sub(r"```$", "", cleaned)  # Remove closing ```
    cleaned = cleaned.strip()

    # Step 3: Extract the first valid JSON object using regex
    match = re.search(r"\{.*\}", cleaned, re.DOTALL)
    if not match:
        raise ValueError("No JSON object found in the response.")

    json_str = match.group(0)

    # Step 4: Parse JSON safely
    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"JSON parsing failed: {e}\nExtracted JSON: {json_str}")



✅ Parsed JSON:
{
  "plan_desc": "This diet plan is designed for Type 2 Diabetes management, focusing on stable blood sugar levels through balanced meals. It incorporates lean proteins, high-fiber complex carbohydrates, and plenty of vegetables while strictly avoiding added sugars and sweets.",
  "add_new_recipe": false,
  "new_recipes": []
}


In [48]:
clean_and_parse_json(response.content)

{'plan_desc': "This daily diet plan is designed for Aarav Mehta, a 29-year-old male with Type 2 Diabetes. The primary goal is to maintain stable blood glucose levels by providing balanced meals with controlled portions of low-glycemic index carbohydrates, lean proteins, and healthy fats, while adhering to a low-sugar and no-sweets restriction. Breakfast consists of 'Gluten-Free Oatmeal Breakfast' to provide slow-release energy. The 'High-Protein Greek Yogurt Parfait' serves as a mid-morning snack to prevent blood sugar dips. Lunch is 'Grilled Chicken with Steamed Vegetables', a low-carb, high-protein meal to avoid post-meal glucose spikes. An evening snack of 'Mixed Nuts and Seeds Portion' is introduced to provide healthy fats and fiber for satiety. Dinner is the 'Diabetic-Friendly Salmon Bowl', which includes omega-3 fatty acids and complex carbohydrates from quinoa for sustained energy and overall cardiovascular health. This plan emphasizes whole foods and consistent meal timing to s